In [11]:
import pandas as pd
import numpy as np
from model import get_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ReLU
from tensorflow.keras.activations import tanh
from tensorflow.keras.layers import BatchNormalization
import keras

In [12]:
rain = pd.read_csv("../Data/Rainfall.csv")
rain.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec,Anomaly
0,1901,34.7,38.6,17.8,38.9,50.6,113.2,241.4,271.6,124.7,52.4,38.7,8.2,1030.8,73.2,107.3,751.0,99.3,-122.62
1,1902,7.4,4.2,19.0,44.1,48.8,111.7,284.9,201.0,200.2,62.5,29.4,25.2,1038.4,11.6,111.9,797.8,117.2,-75.82
2,1903,16.7,8.0,31.1,17.1,59.5,120.3,293.2,274.0,198.1,119.5,40.3,18.0,1195.9,24.7,107.7,885.6,177.8,11.98
3,1904,14.9,9.7,31.4,33.7,73.8,165.5,260.3,207.7,130.8,69.8,11.2,16.4,1025.1,24.5,138.8,764.3,97.4,-109.32
4,1905,24.7,20.3,41.8,33.8,55.8,93.7,253.0,201.7,178.1,54.9,9.6,10.1,977.5,45.0,131.4,726.4,74.7,-147.22


In [13]:
predictor = pd.read_csv("../Data/10_para_Data.csv")
predictor.head()

,Year,NINO_34,EastAsiaSurfacePressure,NorthAtlantaicSurfcePressure,NorthAtlanticOcean_SST,EQ_WIN,AO_oct,PDO_Jun,North_central_Pacific_zonal_Wind,SOI_Jun,SEI_Ocean_SST,ISMR
0,1950,0.365675,-0.386439,-2.546593,0.572334,-0.195580,-0.212910,-2.293911,-9.595640,2.0,0.039908,876.9
1,1951,1.090162,-0.157037,-1.704145,-0.359634,-1.003503,-0.437170,-1.466423,-8.201849,0.2,-0.908400,738.8
2,1952,-0.150832,1.845762,-2.880367,0.585410,-1.739785,-0.194460,-1.982165,-8.985010,0.7,-0.672092,792.9
3,1953,0.347131,1.401160,-2.243482,-0.393405,-1.669605,0.512570,-0.226286,-9.018811,0.1,-0.672323,922.9
4,1954,-1.080947,1.755364,-0.979263,0.021049,0.699660,0.099053,0.475692,-8.474271,0.1,0.012831,885.3


In [14]:
predictor.iloc[0]

Year                                1950.000000
NINO_34                                0.365675
EastAsiaSurfacePressure               -0.386439
NorthAtlantaicSurfcePressure          -2.546593
NorthAtlanticOcean_SST                 0.572334
EQ_WIN                                -0.195580
AO_oct                                -0.212910
PDO_Jun                               -2.293911
North_central_Pacific_zonal_Wind      -9.595640
SOI_Jun                                2.000000
SEI_Ocean_SST                          0.039908
ISMR                                 876.900000
Name: 0, dtype: float64

In [15]:
import glob

slp_path = "../Data/Anomaly/slp/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)
  
slp_months.sort()
print(slp_months)

['../Data/Anomaly/slp\\slp1.csv', '../Data/Anomaly/slp\\slp10.csv', '../Data/Anomaly/slp\\slp11.csv', '../Data/Anomaly/slp\\slp12.csv', '../Data/Anomaly/slp\\slp2.csv', '../Data/Anomaly/slp\\slp3.csv', '../Data/Anomaly/slp\\slp4.csv', '../Data/Anomaly/slp\\slp5.csv', '../Data/Anomaly/slp\\slp6.csv', '../Data/Anomaly/slp\\slp7.csv', '../Data/Anomaly/slp\\slp8.csv', '../Data/Anomaly/slp\\slp9.csv']


In [16]:
#Here i am just storing rainfall data of Jun to sep from 1949 to 2014
# index 47= 1948,
import statistics
lpa = statistics.mean(list(rain['Jun-Sep'][60:89]))
print(lpa)
rain_fall_data = list(rain['Jun-Sep'][47:2020])
rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

878.0689655172414


In [17]:
from scipy.stats import pearsonr
correlation = []

# chacking for each MONTH
month_path = slp_months[1] #0th in dex means january
# reading month data
month_data = pd.read_csv(month_path)
# month_data.head()
# len(month_data)

In [18]:
month_data.iloc[0]
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

# to combine all the features together
combine = []
for i in range(12):
  # chacking for each MONTH
  month_path = slp_months[i] #0th in dex means january
  # reading month data
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:53]  #Here i am using anly data from 1948 to 2000
  for i in feature:
    combine.append(i)
x = np.array(combine)
x.shape

(636, 324)

In [19]:
# define encoder
def get_model_1(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  # encoder level 1
  e = Dense(n_inputs)(visible)
  # e = BatchNormalization()(e)
  e = ReLU()(e)
  

  # bottleneck
  # n_bottleneck = round(float(n_inputs) / 2.0)
  bottleneck = Dense(97)(e)
  # e = BatchNormalization()(bottleneck)

  # decoder level 1
  # d = Dense(n_inputs*2)(bottleneck)
  # d = BatchNormalization()(d)
  # d = LeakyReLU()(d)
  # output layer
  output = Dense(n_inputs, activation='linear')(bottleneck)
  # define autoencoder model
  model = Model(inputs=visible, outputs=output)
  # compile autoencoder model
  model.compile(optimizer='adam', loss='mse')
  # plot the autoencoder
  # plot_model(model, 'autoencoder_compress.png', show_shapes=True)
  return model

In [20]:
model_1 = get_model_1()
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense (Dense)               (None, 324)               105300    
                                                                 
 re_lu (ReLU)                (None, 324)               0         
                                                                 
 dense_1 (Dense)             (None, 97)                31525     
                                                                 
 dense_2 (Dense)             (None, 324)               31752     
                                                                 
Total params: 168,577
Trainable params: 168,577
Non-trainable params: 0
_________________________________________________________________


In [21]:
# fit the autoencoder model to reconstruct input
history = model_1.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/800
40/40 - 0s - loss: 11.5865 - val_loss: 7.3461 - 415ms/epoch - 10ms/step
Epoch 2/800
40/40 - 0s - loss: 6.1429 - val_loss: 4.8097 - 63ms/epoch - 2ms/step
Epoch 3/800
40/40 - 0s - loss: 4.2362 - val_loss: 3.4843 - 63ms/epoch - 2ms/step
Epoch 4/800
40/40 - 0s - loss: 3.2317 - val_loss: 2.7636 - 63ms/epoch - 2ms/step
Epoch 5/800
40/40 - 0s - loss: 2.6480 - val_loss: 2.3088 - 64ms/epoch - 2ms/step
Epoch 6/800
40/40 - 0s - loss: 2.2649 - val_loss: 2.0137 - 92ms/epoch - 2ms/step
Epoch 7/800
40/40 - 0s - loss: 1.9905 - val_loss: 1.7863 - 85ms/epoch - 2ms/step
Epoch 8/800
40/40 - 0s - loss: 1.7890 - val_loss: 1.6039 - 76ms/epoch - 2ms/step
Epoch 9/800
40/40 - 0s - loss: 1.6163 - val_loss: 1.4769 - 64ms/epoch - 2ms/step
Epoch 10/800
40/40 - 0s - loss: 1.4865 - val_loss: 1.3402 - 69ms/epoch - 2ms/step
Epoch 11/800
40/40 - 0s - loss: 1.3752 - val_loss: 1.2526 - 64ms/epoch - 2ms/step
Epoch 12/800
40/40 - 0s - loss: 1.2813 - val_loss: 1.1613 - 72ms/epoch - 2ms/step
Epoch 13/800
40/40 - 0

In [22]:
# define encoder
def get_model_2(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  # encoder level 1
  e = Dense(n_inputs)(visible)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  e = Dense(97)(e)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  # bottleneck
  # n_bottleneck = round(float(n_inputs) / 2.0)
  bottleneck = Dense(29)(e)
  # e = BatchNormalization()(bottleneck)

  e = Dense(97)(bottleneck)
  e = ReLU()(e)
  # e = BatchNormalization()(e)

  # decoder level 1
  # d = Dense(n_inputs*2)(bottleneck)
  # d = BatchNormalization()(d)
  # d = LeakyReLU()(d)
  # output layer
  output = Dense(n_inputs, activation='linear')(e)
  # define autoencoder model
  model = Model(inputs=visible, outputs=output)
  # compile autoencoder model
  model.compile(optimizer='adam', loss='mse')
  # plot the autoencoder
  # plot_model(model, 'autoencoder_compress.png', show_shapes=True)
  return model

In [23]:
model_2 = get_model_2()
model_2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense_3 (Dense)             (None, 324)               105300    
                                                                 
 re_lu_1 (ReLU)              (None, 324)               0         
                                                                 
 dense_4 (Dense)             (None, 97)                31525     
                                                                 
 re_lu_2 (ReLU)              (None, 97)                0         
                                                                 
 dense_5 (Dense)             (None, 29)                2842      
                                                                 
 dense_6 (Dense)             (None, 97)                2910

In [24]:
weight_transfer_layer_model_2 = [1,2,3,8]
weight_transfer_from_layer_model_1 = [1,2,3,4]

for i in range(len(weight_transfer_layer_model_2)):
  model_2_index = weight_transfer_layer_model_2[i]
  model_1_index = weight_transfer_from_layer_model_1[i]
  model_2.layers[model_2_index] = model_1.layers[model_1_index]
  model_2.layers[model_2_index].trainable = False

In [25]:
# fit the autoencoder model to reconstruct input
history = model_2.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/800
40/40 - 0s - loss: 13.2882 - val_loss: 9.7143 - 367ms/epoch - 9ms/step
Epoch 2/800
40/40 - 0s - loss: 8.8320 - val_loss: 7.8621 - 74ms/epoch - 2ms/step
Epoch 3/800
40/40 - 0s - loss: 7.4133 - val_loss: 6.6756 - 53ms/epoch - 1ms/step
Epoch 4/800
40/40 - 0s - loss: 6.2024 - val_loss: 5.5453 - 65ms/epoch - 2ms/step
Epoch 5/800
40/40 - 0s - loss: 5.2768 - val_loss: 4.7633 - 77ms/epoch - 2ms/step
Epoch 6/800
40/40 - 0s - loss: 4.6033 - val_loss: 4.2479 - 55ms/epoch - 1ms/step
Epoch 7/800
40/40 - 0s - loss: 4.1501 - val_loss: 3.9231 - 64ms/epoch - 2ms/step
Epoch 8/800
40/40 - 0s - loss: 3.7950 - val_loss: 3.6073 - 66ms/epoch - 2ms/step
Epoch 9/800
40/40 - 0s - loss: 3.5684 - val_loss: 3.3612 - 66ms/epoch - 2ms/step
Epoch 10/800
40/40 - 0s - loss: 3.3762 - val_loss: 3.1897 - 65ms/epoch - 2ms/step
Epoch 11/800
40/40 - 0s - loss: 3.1884 - val_loss: 3.0143 - 64ms/epoch - 2ms/step
Epoch 12/800
40/40 - 0s - loss: 3.0484 - val_loss: 2.9031 - 73ms/epoch - 2ms/step
Epoch 13/800
40/40 - 0s

In [26]:
# define encoder
def get_model_3(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  # encoder level 1
  e = Dense(n_inputs)(visible)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  e = Dense(97)(e)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  e = Dense(29)(e)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  # bottleneck
  # n_bottleneck = round(float(n_inputs) / 2.0)
  bottleneck = Dense(9)(e)
  # e = BatchNormalization()(bottleneck)

  e = Dense(29)(bottleneck)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  e = Dense(97)(e)
  # e = BatchNormalization()(e)
  e = ReLU()(e)

  # decoder level 1
  # d = Dense(n_inputs*2)(bottleneck)
  # d = BatchNormalization()(d)
  # d = LeakyReLU()(d)
  # output layer
  output = Dense(n_inputs, activation='linear')(e)
  # define autoencoder model
  model = Model(inputs=visible, outputs=output)
  # compile autoencoder model
  model.compile(optimizer='adam', loss='mse')
  # plot the autoencoder
  # plot_model(model, 'autoencoder_compress.png', show_shapes=True)
  return model

In [27]:
model_3 = get_model_3()
model_3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense_8 (Dense)             (None, 324)               105300    
                                                                 
 re_lu_4 (ReLU)              (None, 324)               0         
                                                                 
 dense_9 (Dense)             (None, 97)                31525     
                                                                 
 re_lu_5 (ReLU)              (None, 97)                0         
                                                                 
 dense_10 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_6 (ReLU)              (None, 29)                0   

In [28]:
# fit the autoencoder model to reconstruct input
history = model_3.fit(x, x, epochs=1000, batch_size=16, verbose=2, validation_data=(x,x))
# plot loss

Epoch 1/1000
40/40 - 0s - loss: 13.8024 - val_loss: 10.5361 - 443ms/epoch - 11ms/step
Epoch 2/1000
40/40 - 0s - loss: 9.6054 - val_loss: 8.9460 - 69ms/epoch - 2ms/step
Epoch 3/1000
40/40 - 0s - loss: 8.8451 - val_loss: 8.4887 - 70ms/epoch - 2ms/step
Epoch 4/1000
40/40 - 0s - loss: 8.3602 - val_loss: 8.0125 - 69ms/epoch - 2ms/step
Epoch 5/1000
40/40 - 0s - loss: 7.8734 - val_loss: 7.5495 - 71ms/epoch - 2ms/step
Epoch 6/1000
40/40 - 0s - loss: 7.5096 - val_loss: 7.2688 - 70ms/epoch - 2ms/step
Epoch 7/1000
40/40 - 0s - loss: 7.2940 - val_loss: 7.0925 - 69ms/epoch - 2ms/step
Epoch 8/1000
40/40 - 0s - loss: 7.1594 - val_loss: 6.9910 - 68ms/epoch - 2ms/step
Epoch 9/1000
40/40 - 0s - loss: 7.0234 - val_loss: 6.8924 - 71ms/epoch - 2ms/step
Epoch 10/1000
40/40 - 0s - loss: 6.9121 - val_loss: 6.7778 - 71ms/epoch - 2ms/step
Epoch 11/1000
40/40 - 0s - loss: 6.8262 - val_loss: 6.6628 - 70ms/epoch - 2ms/step
Epoch 12/1000
40/40 - 0s - loss: 6.7141 - val_loss: 6.5515 - 71ms/epoch - 2ms/step
Epoch 13/

In [29]:
model_final1 = Model(inputs=model_3.input, outputs=model_3.layers[1].output)
model_final1.trainable = False

model_final2 = Model(inputs=model_3.input, outputs=model_3.layers[3].output)
model_final2.trainable = False

model_final3 = Model(inputs=model_3.input, outputs=model_3.layers[5].output)
model_final3.trainable = False

model_final4 = Model(inputs=model_3.input, outputs=model_3.layers[7].output)
model_final4.trainable = False

models = [model_final1, model_final2, model_final3, model_final4]
# models = [model_final1, model_final4]

In [30]:
month_data.iloc[0]
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[2:]))
  # print(new_data)
  return np.array(new_data)

# to combine all the features together
def predictor(month,model_final):
  # chacking for each MONTH
  month_path = slp_months[month] #0th in dex means january
  # reading month data
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:]  #Here i am using anly data from 1948 to 2000
  pred = np.array(feature)
  print(pred)
  pred_m = model_final(pred)
  pred_f = pred_m.numpy()
  return pred_f

In [31]:
def get_top_pred(top,pred_info):
  k = 0
  for m in months:
    pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(0,53):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[0:53],score)
    cor_list.append(abs(corr))
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)

In [32]:
# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models):
  slp = predictor(months[0],models[0])
  print(slp.shape)
  pred = get_top_pred(top, slp)
  print(len(pred[0]))
  pred = np.array(pred)
  for mod in models[1:]:
    slp = predictor(months[0],mod)
    b = get_top_pred(top, slp)
    pred = np.concatenate((pred,b), axis=1)
  if(len(months)>1):
    for i in months[1:]:
      for mod in models[0:]:
        slp = predictor(i,mod)
        b = get_top_pred(top, slp)
        pred = np.concatenate((pred,b), axis=1)
  return pred

In [33]:
# pred = get_top_pred(4,pred_f)
from sklearn import preprocessing
months = [0,1,2]
pred = get_predictors(months,5,models)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#This gave 0.76 for month feb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR

tree = DecisionTreeRegressor(max_depth=4)
from sklearn.ensemble import RandomForestRegressor
cor_all = []
reg_f = SVR(kernel = 'rbf')
for i in range(1,6):
  for j in range(40):
    reg = BaggingRegressor( n_estimators=i, random_state=j).fit(pred[:53], rain_fall_data[:53])
    regr = RandomForestRegressor(max_depth=i, random_state=j)
    reg_f.fit(pred[:53], rain_fall_data[:53])
    preds = reg_f.predict(pred[53:67])
    corr, _ = pearsonr(rain_fall_data[53:67],preds)
    cor_all.append(corr)
    print("depth ",i,"    ",corr)
print(min(cor_all),"   ",max(cor_all))

depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      0.40944898117053136
depth  1      

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#w_start = window start, w_end = window end  
def get_top_pred(top,pred_info,w_start,w_end):
  k = 0
  for m in months:
    pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(w_start,w_end):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
    cor_list.append(corr)
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)


# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models,w_start,w_end):
  slp = predictor(months[0],models[0])
  # print(slp.shape)
  pred = get_top_pred(top, slp, w_start,w_end)
  # print(len(pred[0]))
  pred = np.array(pred)
  for mod in models[1:]:
    slp = predictor(months[0],mod)
    b = get_top_pred(top, slp,w_start,w_end)
    pred = np.concatenate((pred,b), axis=1)
  if(len(months)>1):
    for i in months[1:]:
      for mod in models[0:]:
        slp = predictor(i,mod)
        b = get_top_pred(top, slp, w_start,w_end)
        pred = np.concatenate((pred,b), axis=1)
  return pred

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  # reg = make_pipeline(StandardScaler(), SVR(kernel = 'rbf',C=1.0, epsilon=0.2))
  # months = [0]
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,models,53-window,53)
    for i in range(14):
      start = 53-window
      end = 53+1
      # pred = get_predictors(months,2,models,start,end)
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      print(len(pred))
      score.append(reg.predict([pred[53+i]])[0])
    # print(score)
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)
#This is by taking correlation after every iteraion of each predict

In [ ]:
months_comb = [[4]]
top = 9
models = [model_final3 ]
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print(" Month combination = ",month,"    top = ",i,"   min and max",left,"   ",right)

[[ 1.52601400e+00  1.52601400e+00  1.52601400e+00 ...  2.07934933e+00
   2.47264167e+00  1.70730300e+00]
 [ 8.60601400e+00  8.60601400e+00  8.60601400e+00 ...  6.99349333e-01
   1.88268167e+00  1.62734300e+00]
 [ 1.13660140e+01  1.13660140e+01  1.13660140e+01 ...  2.60934933e+00
   2.01268167e+00 -5.27070000e-02]
 ...
 [-6.84016000e-01 -6.84016000e-01 -6.84016000e-01 ... -2.80650667e-01
  -7.73183333e-02  1.29734300e+00]
 [-6.72398600e+00 -6.72398600e+00 -6.72398600e+00 ... -5.40650667e-01
   2.68166667e-03  4.04734300e+00]
 [-3.03986000e-01 -3.03986000e-01 -3.03986000e-01 ...  1.53934933e+00
   2.00268167e+00  4.23734300e+00]]


ValueError: Input 0 of layer "model_5" is incompatible with the layer: expected shape=(None, 324), found shape=(73, 323)